# Modeling

In [1]:
import re #Regular Expression package
import pandas as pd #Pandas dataframes and etc.
import numpy as np #NUmpy package
import matplotlib.pyplot as plt #Matplotlib graphing package
%matplotlib inline
# from sklearn.manifold import TSNE
import nltk #NLTK NLP Package
from nltk.tokenize import word_tokenize #NLTK NLP Package
np.random.seed(0)
from nltk.corpus import gutenberg, stopwords #NLTK NLP Package
from nltk.collocations import * #NLTK NLP Package
from nltk import FreqDist #NLTK NLP Package
from nltk import word_tokenize #NLTK NLP Package
from nltk.stem.porter import * #NLTK Stemming
from nltk.stem import * #NLTK Lemmatization
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split #Sklearn modeling etc. 
from sklearn.feature_extraction.text import CountVectorizer #Sklearn modeling etc.
from sklearn.linear_model import LogisticRegression #Sklearn modeling etc.
import seaborn as sns #Seaborn
from keras.preprocessing.text import Tokenizer #Keras Tokenizer
from keras.preprocessing.sequence import pad_sequences #Keras padding
from keras.models import Sequential #Keras Sequential Decl.
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D #Neural Networks
from keras.utils.np_utils import to_categorical #Encoding labeling
from keras.callbacks import EarlyStopping #Keras early stopback
from keras.layers import Dropout #Dropout
import re #Regular Expression
import string #String package

Using TensorFlow backend.


### Baseline Model

In [2]:
df=pd.read_csv('January-October')

In [3]:
stopwords_list = stopwords.words('english') #Stop words from english language
stopwords_list += list(string.punctuation) #String characters 
stopwords_list += ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',"’", '‘','said','says', "'s",'”','“'] #Miscelleneous and numerical characters 

In [4]:
stopwords_lowercase=['i','me','my','myself','we','our','ours','ourselves',
 'you', "you're", "you've", "you'll","you'd", 'your','yours', 'yourself',
'yourselves', 'he','him','his', 'himself','she', "she's",
 'her', 'hers','herself','it', "it's",'its', 'itself','they', 'them','their', 'theirs',
 'themselves','what', 'which','who', 'whom','this', 'that',
 "that'll", 'these','those', 'am','is', 'are','was', 'were',
 'be', 'been','being', 'have', 'has', 'had','having', 'do',
 'does', 'did','doing', 'a', 'an', 'the','and', 'but',
 'if', 'or','because','as','until', 'while','of','at','by', 'for','with','about','against',
 'between','into','through','during', 'before','after','above','below',
 'to','from','up','down', 'in','out','on','off',
 'over','under','again','further', 'then','once','here','there',
 'when','where','why','how', 'all','any','both','each', 'few','more',
 'most','other', 'some','such', 'no','nor', 'not','only',
 'own','same', 'so','than', 'too','very', 's','t',
 'can','will','just','don', "don't",'should', "should've",'now',
 'd','ll', 'm','o', 're','ve', 'y','ain',
 'aren',"aren't", 'couldn',"couldn't", 'didn',"didn't", 'doesn',"doesn't",
 'hadn',"hadn't", 'hasn',"hasn't", 'haven',"haven't", 'isn',"isn't",
 'ma','mightn', "mightn't",'mustn', "mustn't",'needn', "needn't",'shan',
 "shan't",'shouldn',"shouldn't",'wasn',"wasn't",
 'weren',"weren't", 'won',"won't", 'wouldn',"wouldn't"]

In [5]:
for x in stopwords_lowercase: #Decided to include a for loop to capatilize the stop words due to capitalization in headings. 
    stopwords_list.append(x.capitalize())

In [6]:
tokenized_df=df['Text'].apply(nltk.word_tokenize)
df_tokens=[] 
for x in tokenized_df: #Stop word removal along with lower casing the tokens
    for y in x:
        if y not in stopwords_list:
            df_tokens.append(y.lower())

In [7]:
X=df['Text'] #Splitting into X and y variables
y=df['Coded']

sent_train, sent_test, y_train, y_test = train_test_split(X,y, test_size=.25, random_state=1) #Train test split

In [8]:
vectorizer= CountVectorizer() #Vectorization object
vectorizer.fit(sent_train)

X_train= vectorizer.transform(sent_train) #transforming vectors
X_test= vectorizer.transform(sent_test) #transforming vectors

In [9]:
classifier = LogisticRegression() #Classifier
classifier.fit(X_train, y_train) #Fitting model
score = classifier.score(X_test, y_test) #Accuracy Score

print("Accuracy:", score)

C:\Users\GBLS\AppData\Local\Continuum\anaconda3\New folder\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy: 0.6455203116304953


Basic tokenization was used and achieved accuracy of 64 percent. The two paths forward are as follow: make the model more complex or tokenize the words differently by using bigrams or lemmentization. Hopefully, by isolating the models and only tweaking one aspect of the data/moddel we will improve our accuracy.

First I am going to try to change the tokenized inputs of the models:
    1. Stemming and Lemmentization
    2. Bigrams

## Stemming and Lemmentization

In [10]:
df.head() #Loading original dataframe

,Date,Text,Change,Coded
0,2019-01-01,Trump Invites Top Lawmakers in Effort To End S...,1.0,1.0
1,2019-01-01,Kim Jong Un Extends Peace Overture to U.S.,1.0,1.0
2,2019-01-01,"American Detained in Russia Isn’t a Spy, Famil...",1.0,1.0
3,2019-01-01,The Money Managers to Watch in 2019,1.0,1.0
4,2019-01-01,Investors Try Not to Panic Over Stock Volatility,1.0,1.0


In [11]:
stemmer = PorterStemmer() #Creating the PorterStemmer
stemmed_words= [stemmer.stem(token) for token in df_tokens] #Stemming the tokens

In [12]:
FreqDist(stemmed_words).most_common(10)

[('new', 2688),
 ('u.s.', 2493),
 ('trump', 2073),
 ('compani', 1346),
 ('million', 1261),
 ('year', 1141),
 ('presid', 1089),
 ('say', 1088),
 ('report', 1082),
 ('hous', 1069)]

There really is not a difference between the stemmed words vs. the traditional tokenized word counts. Now lets run a model to see if this had any effect on the modeling accuracy.

In [13]:
def stemming_tokenizer(str_input): #Defining a function to be able to load this into our model as the tokenizer
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    words = [PorterStemmer().stem(word) for word in words]
    return words

In [14]:
X=df['Text']
y=df['Coded']

sent_train, sent_test, y_train, y_test = train_test_split(X,y, test_size=.25, random_state=1)

vectorizer= CountVectorizer(tokenizer=stemming_tokenizer) #Everything else is the same as passed models, except tweaking tokenizer
vectorizer.fit(sent_train)

X_train= vectorizer.transform(sent_train)
X_test= vectorizer.transform(sent_test)

In [15]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", round(score*100,2),'%')

Accuracy: 64.92 %


As we see here, stemming did not improve our model by very much. Essentially it improved by a little under a percentage point. 

In [16]:
def lemmatizeration(x):
    for x in df_tokens:
        word= []
        lemmatizer = WordNetLemmatizer() 
        word.append(lemmatizer.lemmatize(x))
        return word

In [17]:
X=df['Text']
y=df['Coded']

sent_train, sent_test, y_train, y_test = train_test_split(X,y, test_size=.25, random_state=1)

vectorizer= CountVectorizer(tokenizer=lemmatizeration)
vectorizer.fit(sent_train)

X_train= vectorizer.transform(sent_train)
X_test= vectorizer.transform(sent_test)

In [18]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", round(score*100,2),'%')

Accuracy: 64.93 %


Lemmatizing the text gave us a identical result to stemming. I am not confident that changing the mode of text will help the model, but I will try one last strategy, Bigrams. 

## Bigrams

Another strategy for tokenizing is using bigrams, which are words that essentailly belong or show up together in sentances. The computer is able to recongnize which words belong together and which ones do not. Below is the analysis of the bigrams. 

In [19]:
bigram_measures = nltk.collocations.BigramAssocMeasures() #Bigrams created and scored
wsj_finder = BigramCollocationFinder.from_words(df_tokens)
wsj_scored = wsj_finder.score_ngrams(bigram_measures.raw_freq)
wsj_scored[:50]

[(('president', 'trump'), 0.0017477505802716874),
 (('new', 'york'), 0.0016830190772986618),
 (('logistics', 'report'), 0.0009216533042349815),
 (('today', 'logistics'), 0.0009216533042349815),
 (('trump', 'administration'), 0.0009154883991899314),
 (('hong', 'kong'), 0.0008939112315322563),
 (('white', 'house'), 0.0008939112315322563),
 (('wall', 'street'), 0.0006473150297302546),
 (('morning', 'risk'), 0.0006010782418923793),
 (('risk', 'report'), 0.0006010782418923793),
 (('737', 'max'), 0.0005579239065770289),
 (('morning', 'download'), 0.0004901099510814785),
 (('york', 'city'), 0.0004746976884688534),
 (('supreme', 'court'), 0.0004407907107210781),
 (('prime', 'minister'), 0.0004315433531535031),
 (('great', 'escapes'), 0.000422295995585928),
 (('justice', 'department'), 0.00040380128045077785),
 (('last', 'year'), 0.0003791416602705777),
 (('north', 'korea'), 0.0003760592077480527),
 (('paid', 'program'), 0.00037297675522552764),
 (('pg', 'e'), 0.0003575644926129025),
 (('federa

PMI score: measure of association used in information theory and statistics referring to single events.

Below are the calculations and code for this. It is very interesting due to the fact that it can call out important events that have happened over the year. 

In [23]:
wsj_pmi_finder = BigramCollocationFinder.from_words(df_tokens) #PMI Score creation
wsj_pmi_finder.apply_freq_filter(5)
wsj_pmi_scored = wsj_pmi_finder.score_ngrams(bigram_measures.pmi)
wsj_pmi_scored[:10]

[(('abdelaziz', 'bouteflika'), 15.985561800747938),
 (('clayton', 'kershaw'), 15.985561800747938),
 (('edith', 'wharton'), 15.985561800747938),
 (('f.h', 'bertling'), 15.985561800747938),
 (('fire-', 'paparazzi-proofed'), 15.985561800747938),
 (('harriet', 'tubman'), 15.985561800747938),
 (('history—and', 'odds—are'), 15.985561800747938),
 (('javad', 'zarif'), 15.985561800747938),
 (('johns', 'hopkins'), 15.985561800747938),
 (('karlie', 'kloss'), 15.985561800747938)]

1. Abdelaziz Bouteflika: He was the President of Algeria and in 2019 he was removed from office by stepping down due to mass protests. 
2. Clayton Kershaw: A pitcher for the Los Angeles Dodgers who has struggled as of late in the post season. 
3. Nipsey Hussel: Famous Compton rapper who was killed in broad day light outside of his Marathon store. This was a huge deal for the rap and music community and touched many peopls hearts. 
4. Achilles Temdpm: Most likely in reference to the many athletes over 2019 that had achilles injuries. Just to name a few: John Wall, Kevin Durant, Dez Bryant

### Modeling with Bigrams

In [45]:
X=df['Text'] #X and y declared
y=df['Coded']

sent_train, sent_test, y_train, y_test = train_test_split(X,y, test_size=.25, random_state=1) #Train test split

vectorizer= CountVectorizer(ngram_range=(2,2), stop_words='english') #Passing in bigram arguement
vectorizer.fit(sent_train)

X_train= vectorizer.transform(sent_train)
X_test= vectorizer.transform(sent_test)

In [46]:
classifier = LogisticRegression() #Defining and fiting the classifier
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", round(score*100,2),'%')

Accuracy: 69.08 %


The bigrams did in fact improve our model by around 4 or 5 percent! The next step is to use neural networks and other more advanced modeling techniques. The following sections will focus on this. I am hoping to get at least a 5-10 percent increase in accuracy.

## Neural Network 

The highest accuracy so far came from our last model, using bigrams and logistic regression. I want to try to implement a bit more complex models in the hope to improve our accruacy substantially. First I will start with a basic neural network.

In [47]:
df=pd.read_csv('January-October') #loading Data

In [48]:
MAX_NB_WORDS = 5000 # Max number of word
MAX_SEQUENCE_LENGTH = 250 # This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True) #Tokenizer intialization
tokenizer.fit_on_texts(df['Text'])
word_index = tokenizer.word_index #Tokenizing into index
print('Found %s unique tokens.' % len(word_index))

Found 27479 unique tokens.


In [49]:
X = tokenizer.texts_to_sequences(df['Text']) #Putting data into data tensor with padding to be able to feed into model
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (35938, 250)


In [50]:
Y = pd.get_dummies(df['Coded']) #one hot dummy encoding for y labeling purposes 
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (35938, 2)


In [51]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42) #train test split
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(28750, 250) (28750, 2)
(7188, 250) (7188, 2)


In [52]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 100)          500000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 580,602
Trainable params: 580,602
Non-trainable params: 0
_________________________________________________________________
None


In [53]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

Train on 25875 samples, validate on 2875 samples
Epoch 1/5
25875/25875 [==============================] - 153s 6ms/step - loss: 0.6462 - acc: 0.6520 - val_loss: 0.6493 - val_acc: 0.6417
Epoch 2/5
25875/25875 [==============================] - 159s 6ms/step - loss: 0.6094 - acc: 0.6709 - val_loss: 0.6575 - val_acc: 0.6379
Epoch 3/5
25875/25875 [==============================] - 158s 6ms/step - loss: 0.5573 - acc: 0.7123 - val_loss: 0.6817 - val_acc: 0.6217
Epoch 4/5
25875/25875 [==============================] - 159s 6ms/step - loss: 0.5164 - acc: 0.7435 - val_loss: 0.7072 - val_acc: 0.6170
Epoch 5/5
25875/25875 [==============================] - 165s 6ms/step - loss: 0.4785 - acc: 0.7705 - val_loss: 0.7632 - val_acc: 0.6197


In [54]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

7188/7188 [==============================] - 10s 1ms/step
Test set
  Loss: 0.744
  Accuracy: 0.628


We see that we have a very high loss which means we are overfitting as the training data has max accuracy of 77 percent but the highest valadation accuracy is 64%. I am going to reduce the max number of words to try to narrow down our data to the most imporant words. 

In [55]:
MAX_NB_WORDS=2500 #Reducing max numbder of words

In [56]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True) #Rerunning to fit ot model
tokenizer.fit_on_texts(df['Text'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 27479 unique tokens.


In [57]:
X = tokenizer.texts_to_sequences(df['Text']) #Rerunning for the lesser max_NB_WORDS variable
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)
Y = pd.get_dummies(df['Coded'])
print('Shape of label tensor:', Y.shape)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

Shape of data tensor: (35938, 250)
Shape of label tensor: (35938, 2)
(28750, 250) (28750, 2)
(7188, 250) (7188, 2)


In [58]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 250, 100)          250000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 808       
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 18        
Total params: 331,226
Trainable params: 331,226
Non-trainable params: 0
_________________________________________________________________
None


In [59]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

Train on 25875 samples, validate on 2875 samples
Epoch 1/5
25875/25875 [==============================] - 187s 7ms/step - loss: 0.6481 - acc: 0.6513 - val_loss: 0.6543 - val_acc: 0.6390
Epoch 2/5
25875/25875 [==============================] - 196s 8ms/step - loss: 0.6359 - acc: 0.6528 - val_loss: 0.6517 - val_acc: 0.6390
Epoch 3/5
25875/25875 [==============================] - 175s 7ms/step - loss: 0.6100 - acc: 0.6692 - val_loss: 0.6655 - val_acc: 0.6242
Epoch 4/5
25875/25875 [==============================] - 157s 6ms/step - loss: 0.5861 - acc: 0.6876 - val_loss: 0.6707 - val_acc: 0.6092
Epoch 5/5
25875/25875 [==============================] - 157s 6ms/step - loss: 0.5675 - acc: 0.7048 - val_loss: 0.6872 - val_acc: 0.5970


In [60]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

7188/7188 [==============================] - 9s 1ms/step
Test set
  Loss: 0.677
  Accuracy: 0.616


We see that this substantially brought down the loss score to .687 which is around .244 lower. This is a step in a right direction. I would like to add another layer to see how this affects the metrics. 

In [61]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 250, 100)          250000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_4 (Dense)              (None, 16)                1616      
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 18        
Total params: 332,170
Trainable params: 332,170
Non-trainable params: 0
_________________________________________________________________
None


In [62]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

Train on 25875 samples, validate on 2875 samples
Epoch 1/5
25875/25875 [==============================] - 208s 8ms/step - loss: 0.6468 - acc: 0.6521 - val_loss: 0.6518 - val_acc: 0.6390
Epoch 2/5
25875/25875 [==============================] - 203s 8ms/step - loss: 0.6258 - acc: 0.6576 - val_loss: 0.6562 - val_acc: 0.6259
Epoch 3/5
25875/25875 [==============================] - 208s 8ms/step - loss: 0.5971 - acc: 0.6813 - val_loss: 0.6603 - val_acc: 0.6174
Epoch 4/5
25875/25875 [==============================] - 208s 8ms/step - loss: 0.5781 - acc: 0.6988 - val_loss: 0.6852 - val_acc: 0.6197
Epoch 5/5
25875/25875 [==============================] - 209s 8ms/step - loss: 0.5587 - acc: 0.7139 - val_loss: 0.6852 - val_acc: 0.6235


In [63]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

7188/7188 [==============================] - 10s 1ms/step
Test set
  Loss: 0.678
  Accuracy: 0.631


This did not help either of our metrics and ended up decreasing the accuracy and increasing the loss slightly. 

In [64]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(24, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 250, 100)          250000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_7 (Dense)              (None, 32)                3232      
_________________________________________________________________
dense_8 (Dense)              (None, 24)                792       
_________________________________________________________________
dense_9 (Dense)              (None, 16)                400       
_________________________________________________________________
dense_10 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_11 (Dense)             (None, 2)                

In [65]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

Train on 25875 samples, validate on 2875 samples
Epoch 1/5
25875/25875 [==============================] - 222s 9ms/step - loss: 0.6477 - acc: 0.6521 - val_loss: 0.6531 - val_acc: 0.6390
Epoch 2/5
25875/25875 [==============================] - 223s 9ms/step - loss: 0.6274 - acc: 0.6559 - val_loss: 0.6542 - val_acc: 0.6280
Epoch 3/5
25875/25875 [==============================] - 221s 9ms/step - loss: 0.6016 - acc: 0.6776 - val_loss: 0.6695 - val_acc: 0.6216
Epoch 4/5
25875/25875 [==============================] - 223s 9ms/step - loss: 0.5787 - acc: 0.7000 - val_loss: 0.6715 - val_acc: 0.6146
Epoch 5/5
25875/25875 [==============================] - 221s 9ms/step - loss: 0.5548 - acc: 0.7184 - val_loss: 0.7010 - val_acc: 0.6219


In [66]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

7188/7188 [==============================] - 10s 1ms/step
Test set
  Loss: 0.693
  Accuracy: 0.630


Adjusting the LSTM

In [67]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(LSTM(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 250, 100)          250000    
_________________________________________________________________
lstm_5 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dense_12 (Dense)             (None, 8)                 2056      
_________________________________________________________________
dense_13 (Dense)             (None, 2)                 18        
Total params: 617,642
Trainable params: 617,642
Non-trainable params: 0
_________________________________________________________________
None


In [68]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

Train on 25875 samples, validate on 2875 samples
Epoch 1/5
25875/25875 [==============================] - 553s 21ms/step - loss: 0.6468 - acc: 0.6525 - val_loss: 0.6511 - val_acc: 0.6390
Epoch 2/5
25875/25875 [==============================] - 565s 22ms/step - loss: 0.6213 - acc: 0.6631 - val_loss: 0.6554 - val_acc: 0.6310
Epoch 3/5
25875/25875 [==============================] - 564s 22ms/step - loss: 0.5938 - acc: 0.6850 - val_loss: 0.6608 - val_acc: 0.6174
Epoch 4/5
25875/25875 [==============================] - 563s 22ms/step - loss: 0.5933 - acc: 0.6861 - val_loss: 0.7001 - val_acc: 0.6137
Epoch 5/5
25875/25875 [==============================] - 564s 22ms/step - loss: 0.5682 - acc: 0.7076 - val_loss: 0.6891 - val_acc: 0.6322


In [69]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

7188/7188 [==============================] - 60s 8ms/step
Test set
  Loss: 0.679
  Accuracy: 0.640


Trial and error with tweaking the model, changing the word counts and also adding layers.

In [70]:
MAX_NB_WORDS=2500
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['Text'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
X = tokenizer.texts_to_sequences(df['Text'])
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)
Y = pd.get_dummies(df['Coded'])
print('Shape of label tensor:', Y.shape)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

Found 27479 unique tokens.
Shape of data tensor: (35938, 250)
Shape of label tensor: (35938, 2)
(28750, 250) (28750, 2)
(7188, 250) (7188, 2)


In [71]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(LSTM(120, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(8, activation='sigmoid'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 250, 100)          250000    
_________________________________________________________________
lstm_6 (LSTM)                (None, 120)               106080    
_________________________________________________________________
dense_14 (Dense)             (None, 8)                 968       
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 18        
Total params: 357,066
Trainable params: 357,066
Non-trainable params: 0
_________________________________________________________________
None


In [72]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

Train on 25875 samples, validate on 2875 samples
Epoch 1/5
25875/25875 [==============================] - 346s 13ms/step - loss: 0.6484 - acc: 0.6478 - val_loss: 0.6535 - val_acc: 0.6390
Epoch 2/5
25875/25875 [==============================] - 528s 20ms/step - loss: 0.6383 - acc: 0.6529 - val_loss: 0.6558 - val_acc: 0.6379
Epoch 3/5
25875/25875 [==============================] - 438s 17ms/step - loss: 0.6163 - acc: 0.6677 - val_loss: 0.6606 - val_acc: 0.6083
Epoch 4/5
25875/25875 [==============================] - 438s 17ms/step - loss: 0.5959 - acc: 0.6879 - val_loss: 0.6688 - val_acc: 0.6108
Epoch 5/5
25875/25875 [==============================] - 463s 18ms/step - loss: 0.5784 - acc: 0.7022 - val_loss: 0.6805 - val_acc: 0.6125


In [73]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

7188/7188 [==============================] - 23s 3ms/step
Test set
  Loss: 0.669
  Accuracy: 0.629


Adding another layer to see if this can improve any of the metrics.

In [74]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(LSTM(120, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(16, activation='sigmoid'))
model.add(Dense(8, activation='sigmoid'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 250, 100)          250000    
_________________________________________________________________
lstm_7 (LSTM)                (None, 120)               106080    
_________________________________________________________________
dense_16 (Dense)             (None, 16)                1936      
_________________________________________________________________
dense_17 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_18 (Dense)             (None, 2)                 18        
Total params: 358,170
Trainable params: 358,170
Non-trainable params: 0
_________________________________________________________________
None


In [75]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

Train on 25875 samples, validate on 2875 samples
Epoch 1/5
25875/25875 [==============================] - 451s 17ms/step - loss: 0.6872 - acc: 0.5922 - val_loss: 0.6548 - val_acc: 0.6390
Epoch 2/5
25875/25875 [==============================] - 490s 19ms/step - loss: 0.6462 - acc: 0.6526 - val_loss: 0.6543 - val_acc: 0.6390
Epoch 3/5
25875/25875 [==============================] - 579s 22ms/step - loss: 0.6459 - acc: 0.6526 - val_loss: 0.6546 - val_acc: 0.6390
Epoch 4/5
25875/25875 [==============================] - 579s 22ms/step - loss: 0.6459 - acc: 0.6526 - val_loss: 0.6543 - val_acc: 0.6390
Epoch 5/5
25875/25875 [==============================] - 459s 18ms/step - loss: 0.6459 - acc: 0.6526 - val_loss: 0.6543 - val_acc: 0.6390


In [76]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

7188/7188 [==============================] - 31s 4ms/step
Test set
  Loss: 0.647
  Accuracy: 0.651


Adding another layer! So far it has little affect on both metrics

In [77]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(LSTM(120, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(24, activation='sigmoid'))
model.add(Dense(16, activation='sigmoid'))
model.add(Dense(8, activation='sigmoid'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 250, 100)          250000    
_________________________________________________________________
lstm_8 (LSTM)                (None, 120)               106080    
_________________________________________________________________
dense_19 (Dense)             (None, 24)                2904      
_________________________________________________________________
dense_20 (Dense)             (None, 16)                400       
_________________________________________________________________
dense_21 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_22 (Dense)             (None, 2)                 18        
Total params: 359,538
Trainable params: 359,538
Non-trainable params: 0
________________________________________________

In [78]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

Train on 25875 samples, validate on 2875 samples
Epoch 1/5
25875/25875 [==============================] - 569s 22ms/step - loss: 0.6474 - acc: 0.6526 - val_loss: 0.6545 - val_acc: 0.6390
Epoch 2/5
25875/25875 [==============================] - 471s 18ms/step - loss: 0.6460 - acc: 0.6526 - val_loss: 0.6543 - val_acc: 0.6390
Epoch 3/5
25875/25875 [==============================] - 553s 21ms/step - loss: 0.6459 - acc: 0.6526 - val_loss: 0.6542 - val_acc: 0.6390
Epoch 4/5
25875/25875 [==============================] - 563s 22ms/step - loss: 0.6452 - acc: 0.6526 - val_loss: 0.6537 - val_acc: 0.6390
Epoch 5/5
25875/25875 [==============================] - 600s 23ms/step - loss: 0.6320 - acc: 0.6526 - val_loss: 0.6554 - val_acc: 0.6390


In [79]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

7188/7188 [==============================] - 32s 4ms/step
Test set
  Loss: 0.650
  Accuracy: 0.651


We see that another layer does not improve our accuracy and really only decreases our loss function. 

# Conclusion

The results of the modeling are as follows:  

__Logistic Regression__  
Baseline:
Accuracy - 64.55%  
Lemmatization:
Accuracy - 64.92%  
Stemming:
Accuracy - 64.92%  
Bigrams:
Accuracy - 69.08%  


__Neural Networks__  
Baseline:  
    - Accuracy: 62.80%  
    - Loss: .744    
Optimal Model:  
    - Accuracy: 65.10%  
    - Loss: .650  


__Best Model:__   
Bigram Logistic Regression: This model achieved around 69 percent accuracy in predicting the direction of the S&P 500 for the following day.  
Neural networks were not sufficient and due to the black box component, the optimal number of layers were not found

__NLP Takeaways from WSJ:__   

Trump’s name was mentioned the most both in negative and positive days  
There was not a real difference between the top words on a positive vs. negative days  
Amazon and Apple were the two companies mentioned the most  

__Further Analysis:__   
Statistical testing to really find out what makes a positive vs. negative day  
Time series modeling of Amazon and Apple to see whether the stock market went up or down when mentioned  
Stacked modeling  
Gather more data from the waybackmachine to see if we can improve the modeling  
Lastly, compare Trump's presidency(2016-2020) to Obamas latter tenure (2012-2016) to compare word distributions. Does the media/ewspapers cover Trump more than any other president in history or was Obama covered just as much.